In [ ]:
'''
O que faz e Por que usar

pandas as pd: biblioteca principal para manipulação tabular (DataFrame).

numpy as np: funções numéricas e gerador aleatório vetorizado (mais rápido que random para arrays).

datetime, timedelta: classes do Python para trabalhar com datas (não são estritamente necessárias aqui porque usamos pd.date_range, mas é comum importar quando se manipula datas manualmente).

random: módulo padrão do Python — nem sempre usado, mas muitas vezes incluído para gerar aleatoriedade com random.seed() ou funções não-vetorizadas.

os: manipulação do sistema de arquivos (criar pastas, juntar caminhos).
'''

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import os

In [ ]:
'''
Seta as seeds (sementes) dos geradores aleatórios para reprodutibilidade:

np.random.seed(42) assegura que todas as chamadas de np.random.* gerem a mesma sequência toda vez que o script for executado.

random.seed(42) faz o mesmo para o gerador do módulo random.

Por que ambos? Porque o código usa (ou poderia usar) tanto numpy.random quanto random. Para reproduzir resultados previsíveis, se considera uma boa prática setar ambas.
'''

np.random.seed(46)
random.seed(46)

In [ ]:
'''Define output_dir como a pasta onde o CSV será salvo (datasets).

os.makedirs(output_dir, exist_ok=True) cria a pasta necessária. Se já existir, não lança erro (exist_ok=True).

Garante que df.to_csv(...) não falhe por pasta inexistente.
'''

output_dir = "datasets"
os.makedirs(output_dir, exist_ok=True)

In [ ]:

'''Define quantos registros (linhas) o DataFrame terá inicialmente.'''

num_registros = 3000

In [ ]:
'''Gera uma sequência diária de datas entre 01/01/2023 e 31/12/2023 (inclusive).

datas é um DatetimeIndex com todos os dias do ano (365 entradas).'''

datas = pd.date_range(start="2023-01-01", end="2023-12-31", freq="D")

In [ ]:
'''Listas de valores possíveis para as colunas categóricas — usadas para amostragem aleatória.'''


produtos = ["Notebook", "Smartphone", "Mouse", "Teclado", "Monitor", "Fone de Ouvido", "Impressora", "Cadeira Gamer"]
categorias = ["Informática", "Acessórios", "Periféricos", "Móveis de Escritório"]
pagamentos = ["Crédito", "Débito", "Pix", "Dinheiro", "Boleto"]

In [ ]:
'''"ID": range(1, num_registros + 1) → cria IDs sequenciais (1..1000).

"Data": np.random.choice(datas, size=num_registros) → escolhe aleatoriamente uma data (do DatetimeIndex) para cada linha. Resultado: coluna de tipo datetime (na maioria dos casos ficará como datetime64[ns] até que se insiram NaN).

"Produto", "Categoria", "Pagamento" usam np.random.choice para escolher categorias aleatórias.

"Quantidade": np.random.randint(1, 10, size=num_registros) → inteiros aleatórios entre 1 e 9 (inclusive o 1, exclusivo 10).

"Preço": np.round(np.random.uniform(50, 5000, size=num_registros), 2) → preços float entre 50 e 5000, arredondados para 2 casas (simula ticket/valor unitário).
Observações de tipo:

Antes de inserir NaN, ID é int, Quantidade int, Preço float, Data datetime, outras object (strings).'''


df = pd.DataFrame({
    "ID": range(1, num_registros + 1),
    "Data": np.random.choice(datas, size=num_registros),
    "Produto": np.random.choice(produtos, size=num_registros),
    "Categoria": np.random.choice(categorias, size=num_registros),
    "Quantidade": np.random.randint(1, 10, size=num_registros),
    "Preço": np.round(np.random.uniform(50, 5000, size=num_registros), 2),
    "Pagamento": np.random.choice(pagamentos, size=num_registros)
})


In [ ]:
'''df.size = número total de células do DataFrame = n_linhas * n_colunas (1000 * 7 = 7000).

n_nulos = 5% de 7000 = 350 (no exemplo com 1000 linhas). Ou seja, serão transformadas em NaN 350 células aleatória'''


nulos_porcentagem = 0.05
n_celulas = df.size
n_nulos = int(n_celulas * nulos_porcentagem)

In [ ]:
'''Para cada uma das 350 iterações:

i é um índice de linha aleatório (0..999).

j é um índice de coluna aleatório (0..6).

df.iat[i, j] = np.nan escreve NaN naquela célula (indexação por posição inteira — muito rápida para leituras/escritas escalares).
Efeitos importantes

Pode atingir qualquer coluna, inclusive ID ou Data. Se NaN for inserido numa coluna de inteiros (ID ou Quantidade), o pandas converte automaticamente essa coluna para float (porque NaN é um float especial em pandas). Isso é esperado e um ponto essencial a se lembrar quando tratar tipos depois.

Inserir NaN na coluna Data fará com que ela deixe de ser datetime64 e passe a ter tipo object ou float (dependendo de quantos NaN), exigindo pd.to_datetime() depois para reconverter.

df.iat usa posições inteiras: é diferente de .loc (rótulo) e .iloc (posição por fatia). .iat é o mais rápido para acessar um único elemento.

Observação de performance

Para 1000 linhas, este loop é OK. Para milhões de linhas, é preferível usar estratégias vetorizadas (por ex. amostrar pares (row,col) sem loop).'''

for _ in range(n_nulos):
    i = np.random.randint(0, df.shape[0])
    j = np.random.randint(0, df.shape[1])
    df.iat[i, j] = np.nan

In [ ]:
'''n_duplicatas = 3% de 1000 = 30 (aprox.).

df.sample(...) seleciona aleatoriamente 30 linhas do DataFrame atual (note que isto é feito após inserir os NaN, então as linhas amostradas podem incluir NaN nas mesmas posições).

pd.concat([df, duplicatas], ignore_index=True) concatena o DataFrame original com as linhas amostradas. ignore_index=True reindexa o DataFrame concatenado com novos índices 0..N-1.
Por que isso mantém o mesmo ID

duplicatas é uma cópia exata das linhas selecionadas — incluindo o valor de ID. Ao concatenar, essas linhas duplicadas carregam o mesmo valor de ID da linha original. Ou seja, você terá IDs repetidos — exatamente o que você pediu para simular duplicação de dados de sistema.
Observações

df.duplicated() (sem argumentos) vai contar linhas iguais em todas as colunas. Como as linhas duplicadas são cópias exatas, elas serão detectadas como duplicatas.

Atenção: se você tivesse reatribuído IDs depois (como em uma versão anterior do script), as duplicatas não seriam reconhecidas por df.duplicated() porque o ID na linha duplicada seria diferente. Aqui, mantemos IDs iguais de propósito.'''

n_duplicatas = int(0.03 * len(df))
duplicatas = df.sample(n=n_duplicatas, random_state=42)
df = pd.concat([df, duplicatas], ignore_index=True)


In [ ]:
'''os.path.join monta o caminho correto para o arquivo (ajuda em compatibilidade entre sistemas).

df.to_csv(..., index=False) salva o DataFrame em CSV sem a coluna de índice do pandas (apenas as colunas reais).'''

output_path = os.path.join(output_dir, "vendas_raw.csv")
df.to_csv(output_path, index=False)

In [ ]:
'''Print verificando se o dataset foi gravado com sucesso e o seu caminho'''

print(f"✅ Dataset gerado com sucesso: {output_path}")

✅ Dataset gerado com sucesso: datasets\vendas_raw.csv


In [ ]:
'''Print para ver o total de registros do dataset gerado'''

print(f"Total de registros: {len(df)}")


Total de registros: 3090


In [ ]:
'''Print para ver quantos valores Duplicados foram inseridos'''

print(f"Duplicatas reais: {df.duplicated().sum()}")

Duplicatas reais: 90


In [ ]:
'''Print para ver o total de valores Nulos inseridos'''

print(f"Nulos totais: {df.isnull().sum().sum()}")

Nulos totais: 1058


In [ ]:
'''Print para ver o resumo do DataFrame (tipos e nulos por coluna)'''

print("\nResumo por coluna:")
print(df.info())


Resumo por coluna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3090 entries, 0 to 3089
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          2934 non-null   float64       
 1   Data        2933 non-null   datetime64[ns]
 2   Produto     2914 non-null   object        
 3   Categoria   2954 non-null   object        
 4   Quantidade  2940 non-null   float64       
 5   Preço       2952 non-null   float64       
 6   Pagamento   2945 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 169.1+ KB
None
